In [1]:
import numpy as np
import os,sys
import pandas as pd
import time

In [2]:
def cellDivisions(tm,td,ts,stem,rl,positions):
    global n
    for dividingCell in range(0,n):
        tc = ts - tm[dividingCell]
        tr = td[dividingCell] - tc
        if (dividingCell-n0)*13.2 > rl and ts == 24:    # Pulse
            M_G2 = np.random.normal(9,6)
            S_noSkip = np.random.normal(88,9)
            S_skip = np.random.normal(91,9)
            G1_noSkip = np.random.normal(22,19)
            G1_skip = np.random.normal(130,19)
            if tr > M_G2+S_noSkip:    # si estoy en G2/M o S la dejo continuar
                if M_G2+S_noSkip <= tr < M_G2+S_noSkip+S_skip:        # en el skip de fase S
                    td[dividingCell] = M_G2+S_noSkip     # la saco del skip
                    tm[dividingCell] = ts
                elif M_G2+S_noSkip+S_skip <= tr < M_G2+S_noSkip+S_skip+G1_noSkip:      # en fase G1
                    td[dividingCell] = td[dividingCell]-S_skip         # acorto el ciclo restando el skip de fase S
                else:     # en el skip de fase G1
                    td[dividingCell] = td[dividingCell]-S_skip
            stem[dividingCell] = True
        if tr < 1/3600:
            tm[dividingCell] = ts
            tm = np.insert(tm,dividingCell,ts)
            tm = tm[:-1]            
            n = n+1
            positions = np.insert(positions,dividingCell,-1)
            if stem[dividingCell]:
                td[dividingCell] = np.random.normal(119, 10)
                td = np.insert(td,dividingCell,np.random.normal(119, 10))
                td = td[:-1]
                stem = np.insert(stem,dividingCell,True)
                stem = stem[:-1]
            else:
                td[dividingCell] = np.random.normal(340, 32)
                td = np.insert(td,dividingCell,np.random.normal(340, 32))
                td = td[:-1]
                stem = np.insert(stem,dividingCell,False)
                stem = stem[:-1]                
    return tm,td,stem,positions

In [3]:
def run(steps,tm,td,stem,rl,positions):
    global n
    outgrowth = []
    tracking = []
    tracking.append(positions)
    for ts in range(0,steps+1,1):
        tm,td,stem,positions = cellDivisions(tm,td,ts,stem,rl,positions)
        outgrowth.append(n-n0)    # Appends the outgrowth for each time
        tracking.append(positions)
    return outgrowth,tracking

In [4]:
root = '/home/ecuracosta/Desktop/MS/Simulations/Phases/Skip-Skip/Only_S/'
os.chdir(root)

start_time = time.time() 

emptySpace = 250
steps = 24*8    # Number of steaps (in hours)
n0 = 205    # Number of cells after amputation
rl = -1045    # Recruitment limit
tv = 31

lattice = n0+emptySpace
folder = 'n0='+str(n0)+'_rl='+str(rl)+'_tv='+str(tv)
os.makedirs(folder)
print('n0:',n0)
print('rl:',rl)
print('tv:',tv)    

for seed in range(0,200):
    
    n = n0
    positions = np.arange(0,n0) 
    tm = np.random.uniform(-(340-2*32),0,lattice)
    td = np.random.normal(340,32,lattice)
    stem = np.zeros(lattice)
    solution,track = run(steps,tm,td,stem,rl,positions)

    d = {"time":np.arange(0,steps+1),"outgrowth":solution}
    df = pd.DataFrame(d)
    outfile = open(str(root)+str(folder)+'/'+'sim_seed='+str(seed)+'.csv', 'a')
    df.to_csv(outfile, sep=',')
    outfile.close()

print("Total time: %s seconds <--" % (time.time() - start_time))

n0: 205
rl: -1045
tv: 31
Total time: 8.11110234260559 seconds <--
